In [1]:
import numpy as np
import pandas as pd
from utils import *

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [5]:
total_solar_weather = np.load('total_solar_weather_pm_2013_2020.npy', allow_pickle=True)

In [11]:
total_solar_weather[0][35064:]

array([0.0, 192.0, -2.4, nan, 0.0, 0.0, 85.0, 4.4, -4.5, 1026.5, 1030.4,
       nan, nan, nan, nan, nan, nan, nan, nan, 720.0, nan, nan, -1.3, nan,
       nan, nan, nan, '경남 진주시', '도시대기', 238133, '상대동(진주)', 2017010101,
       0.002, 0.8, 0.01, 0.024, 60.0, 26.0, '경남 진주시 동진로 279'],
      dtype=object)

In [2]:
locations = config['experiments']['location_used']
columns = config['experiments']['features_used']

In [12]:
for loc in locations:
    temp_data = pd.DataFrame(total_solar_weather[locations.index(loc)][35064:], columns=columns)
    temp_data.to_csv(f"./dataset/total_solar_weather_2017_2020_{loc}.csv", encoding='cp949')
    

PCC value 구하기

In [22]:
total_solar_weather[:, 35064:, 0].shape

(38, 35064)

In [23]:
locations = config['experiments']['location_used']
columns = config['experiments']['features_used']

In [31]:
columns = ['발전률',
 '지점',
 '기온(°C)',    #!
 '강수량(mm)',  #!
 '풍속(m__s)',  #!
 '풍향(16방위)',
 '습도(%)',     #!
 '증기압(hPa)',
 '이슬점온도(°C)',
 '현지기압(hPa)',
 '해면기압(hPa)',
 '일조(hr)',
 '일사(MJ__m2)',
 '적설(cm)',
 '3시간신적설(cm)',
 '전운량(10분위)',
 '중하층운량(10분위)',
 '운형(운형약어)',
 '최저운고(100m )',
 '시정(10m)',
 '지면상태(지면상태코드)',
 '현상번호(국내식)',
 '지면온도(°C)',
 '5cm 지중온도(°C)',
 '10cm 지중온도(°C)',
 '20cm 지중온도(°C)',
 '30cm 지중온도(°C)',
 '지역',
 '망',
 '측정소코드',
 '측정소명',
 '측정일시',
 'SO2',
 'CO',
 'O3',
 'NO2',
 'PM10',
 'PM25',
 '주소']

In [32]:
for col in columns:
    temp_data = pd.DataFrame(total_solar_weather[:, 35064:, columns.index(col)].T, columns=locations)
    temp_data.to_csv(f"./dataset/features/total_area_2017_2020_{col}.csv", encoding='cp949')
    

In [57]:
solar = pd.read_csv('./dataset/features/total_area_2017_2020_발전률.csv', encoding='cp949', index_col=0)

In [58]:
solar = solar.T.dropna().T
solar

,경상대,구미,영동,예천,남제주소내,부산복합자재창고,부산복합1단계,부산신항,신인천소내,영월본부,인천수산정수장,하동공설운동장,하동변전소,하동보건소,하동본부,하동정수장,하동하수처리장,대구,신안
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,0.0,0.0,0.321141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35060,0.0,0.0,0.623640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35061,0.0,0.0,0.649846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35062,0.0,0.0,0.130402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
pd.DataFrame(np.corrcoef(solar.T))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1.000000,0.293046,0.748465,0.903825,0.858770,0.913741,0.915757,0.887895,0.854997,0.883450,0.865082,0.952061,0.943028,0.938426,0.935508,0.926747,0.950085,0.853452,0.885709
1,0.293046,1.000000,0.236913,0.290572,0.252026,0.235431,0.271639,0.280099,0.278878,0.279186,0.289216,0.285749,0.273235,0.323863,0.283436,0.281035,0.303977,0.252167,0.266686
2,0.748465,0.236913,1.000000,0.763570,0.688784,0.692918,0.716281,0.701430,0.739819,0.788786,0.751240,0.736705,0.721686,0.736985,0.727355,0.700181,0.732918,0.733963,0.723283
3,0.903825,0.290572,0.763570,1.000000,0.811723,0.865105,0.862984,0.834540,0.871776,0.926757,0.882212,0.893764,0.879294,0.877782,0.877190,0.877899,0.900562,0.834411,0.857522
4,0.858770,0.252026,0.688784,0.811723,1.000000,0.835914,0.826082,0.828186,0.786828,0.800988,0.784949,0.858200,0.833640,0.835677,0.861496,0.846611,0.840565,0.794877,0.849003
5,0.913741,0.235431,0.692918,0.865105,0.835914,1.000000,0.929024,0.886933,0.808248,0.846818,0.812648,0.913002,0.902511,0.836497,0.903552,0.881132,0.897464,0.832066,0.852471
6,0.915757,0.271639,0.716281,0.862984,0.826082,0.929024,1.000000,0.881035,0.817656,0.859132,0.829117,0.905172,0.896542,0.885066,0.891512,0.869723,0.907291,0.815101,0.841891
7,0.887895,0.280099,0.701430,0.834540,0.828186,0.886933,0.881035,1.000000,0.803552,0.818584,0.799848,0.881840,0.857176,0.855920,0.886035,0.875424,0.857158,0.806746,0.833768
8,0.854997,0.278878,0.739819,0.871776,0.786828,0.808248,0.817656,0.803552,1.000000,0.886162,0.954854,0.849584,0.823212,0.838562,0.836287,0.840034,0.841996,0.791041,0.833259
9,0.883450,0.279186,0.788786,0.926757,0.800988,0.846818,0.859132,0.818584,0.886162,1.000000,0.898910,0.876591,0.850783,0.855255,0.858623,0.849134,0.874283,0.827133,0.848955


In [72]:
temp = pd.read_csv('./dataset/features/total_area_2017_2020_기온(°C).csv', encoding='cp949', index_col=0)

In [73]:
temp = temp.interpolate()

In [75]:
temp = temp.T.dropna().T

In [76]:
pd.DataFrame(np.corrcoef(temp.T))

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,1.000000,0.977791,0.979072,1.000000,0.961440,0.968905,0.940440,0.960504,0.960504,0.960504,...,0.962609,0.962609,0.927734,0.961440,0.977791,0.970364,0.934446,0.954855,0.977323,0.971776
1,0.977791,1.000000,0.976970,0.977791,0.973406,0.970508,0.965760,0.982495,0.982495,0.982495,...,0.970200,0.970200,0.950860,0.973406,1.000000,0.979217,0.963492,0.975422,0.979768,0.985967
2,0.979072,0.976970,1.000000,0.979072,0.972191,0.987593,0.933055,0.957888,0.957888,0.957888,...,0.967017,0.967017,0.942775,0.972191,0.976970,0.985517,0.950377,0.961219,0.991417,0.978470
3,1.000000,0.977791,0.979072,1.000000,0.961440,0.968905,0.940440,0.960504,0.960504,0.960504,...,0.962609,0.962609,0.927734,0.961440,0.977791,0.970364,0.934446,0.954855,0.977323,0.971776
4,0.961440,0.973406,0.972191,0.961440,1.000000,0.972555,0.948915,0.962126,0.962126,0.962126,...,0.990810,0.990810,0.947243,1.000000,0.973406,0.987716,0.986556,0.976381,0.966594,0.981504
5,0.968905,0.970508,0.987593,0.968905,0.972555,1.000000,0.928014,0.954619,0.954619,0.954619,...,0.966720,0.966720,0.945307,0.972555,0.970508,0.980910,0.950902,0.956260,0.983428,0.972228
6,0.940440,0.965760,0.933055,0.940440,0.948915,0.928014,1.000000,0.971075,0.971075,0.971075,...,0.951199,0.951199,0.932681,0.948915,0.965760,0.944450,0.951846,0.967357,0.938057,0.958421
7,0.960504,0.982495,0.957888,0.960504,0.962126,0.954619,0.971075,1.000000,1.000000,1.000000,...,0.959809,0.959809,0.958005,0.962126,0.982495,0.962203,0.959628,0.968059,0.966283,0.967987
8,0.960504,0.982495,0.957888,0.960504,0.962126,0.954619,0.971075,1.000000,1.000000,1.000000,...,0.959809,0.959809,0.958005,0.962126,0.982495,0.962203,0.959628,0.968059,0.966283,0.967987
9,0.960504,0.982495,0.957888,0.960504,0.962126,0.954619,0.971075,1.000000,1.000000,1.000000,...,0.959809,0.959809,0.958005,0.962126,0.982495,0.962203,0.959628,0.968059,0.966283,0.967987
